In [ ]:
# SKIP THIS IF YOU WANT TO USE THE NON WEB INTERFACE (can only be done when notebook run locally)
%matplotlib inline

In [ ]:
#%%
try:
    get_ipython().magic(u'load_ext autoreload')
    get_ipython().magic(u'autoreload 2')
    print(1)
except:
    print('NOT IPYTHON')

import sys
import numpy as np
import tifffile
import psutil
from scipy.ndimage.filters import gaussian_filter
from ipyparallel import Client

import caiman as cm
from caiman.utils.visualization import nb_view_patches3d
import caiman.source_extraction.cnmf as cnmf
from caiman.source_extraction.cnmf.utilities import order_components

import matplotlib.pyplot as plt
import bokeh.plotting as bpl
bpl.output_notebook()

In [ ]:
#%%
n_processes = psutil.cpu_count()
print('using ' + str(n_processes) + ' processes')
print("Stopping  cluster to avoid unnencessary use of memory....")
sys.stdout.flush()  
cm.stop_server()

In [ ]:
def gen_data(p = 1, noise=1., T=256, framerate=30, firerate=2., plot=False):
    if p==2:
        gamma = np.array([1.5,-.55])
    elif p==1:
        gamma = np.array([.9])
    else: raise
    dims = (30, 40, 50)  # size of image
    sig = (2, 2, 2)  # neurons size
    bkgrd = 10
    N = 20  # number of neurons
    np.random.seed(7)
    centers = np.asarray([[np.random.randint(5,x-5) 
                           for x in dims] for i in range(N)])
    Yr = np.zeros(dims + (T,), dtype=np.float32)
    trueSpikes = np.random.rand(N,T) < firerate / float(framerate)
    trueSpikes[:,0] = 0
    truth = trueSpikes.astype(np.float32)
    for i in range(2,T):
        if p ==2:
            truth[:,i] += gamma[0]*truth[:,i-1] + gamma[1]*truth[:,i-2] 
        else:
            truth[:,i] += gamma[0]*truth[:,i-1]
    for i in range(N):
        Yr[centers[i,0],centers[i,1],centers[i,2]] = truth[i]
    tmp=np.zeros(dims)
    tmp[15,20,25]=1.
    z=np.linalg.norm(gaussian_filter(tmp, sig).ravel())
    Yr = bkgrd + noise * np.random.randn(*(dims + (T,))) + 10 *gaussian_filter(Yr, sig + (0,)) /z
    d1,d2,d3,T=Yr.shape
    Yr=np.reshape(Yr,(d1*d2*d3,T),order='F').astype(np.float32)
 
    if plot:
        Y=np.reshape(Yr,(d1,d2,d3,T),order='F')
        plt.figure(figsize=(15,3))
        plt.plot(truth.T);
        plt.figure(figsize=(15,3))
        for c in centers:
            plt.plot(Y[c[0],c[1],c[2]]);

        plt.figure(figsize=(15,4))
        plt.subplot(131)
        plt.scatter(*centers.T[::-1],c='g')
        plt.imshow(Y.max(0).max(-1),cmap='hot');plt.title('Max.proj. x & t')
        plt.subplot(132)
        plt.scatter(*centers.T[[2,0,1]],c='g')
        plt.imshow(Y.max(1).max(-1),cmap='hot');plt.title('Max.proj. y & t')
        plt.subplot(133)
        plt.scatter(*centers.T[[1,0,2]],c='g')
        plt.imshow(Y.max(2).max(-1),cmap='hot');plt.title('Max.proj. z & t')
        plt.show()
        
    return Yr, truth, trueSpikes, centers, dims

In [ ]:
plt.close('all')
#%% SAVING TIFF FILE ON A SINGLE MEMORY MAPPABLE FILE
try:
    fname_new=cm.save_memmap(['example_movies/demoMovie3D.tif'],base_name='Yr',is_3D=True)
except: #%% create 3d tiff file if not yet existent
    Yr, truth, trueSpikes, centers, dims = gen_data(p=2)
    # imagej takes data shapes up to 6 dimensions in TZCYXS order
    data = np.transpose(Yr.reshape(dims+(-1,1), order='F'), [3, 0, 4, 1, 2])
    t = tifffile.TiffWriter('example_movies/demoMovie3D.tif', imagej=True)
    t.save(data)
    t.close()
    fname_new=cm.save_memmap(['example_movies/demoMovie3D.tif'],base_name='Yr',is_3D=True)

print(fname_new)

In [ ]:
Yr,dims,T=cm.load_memmap(fname_new)
Y=np.reshape(Yr,dims+(T,),order='F')
Cn = cm.local_correlations(Y)
plt.imshow(Cn.max(0) if len(Cn.shape)==3 else Cn, cmap='gray',
           vmin=np.percentile(Cn, 1), vmax=np.percentile(Cn, 99))    
plt.show()

In [ ]:
# set parameters
K=20 # number of neurons expected per patch
gSig=[2,2,2] # expected half size of neurons
merge_thresh=0.8 # merging threshold, max correlation allowed
p=2 #order of the autoregressive system

In [ ]:
# START CLUSTER
cm.start_server()
c=Client()

In [ ]:
# INIT
cnm = cnmf.CNMF(n_processes, method_init='greedy_roi', k=K, gSig=gSig, merge_thresh=merge_thresh,
                p=p, dview=c[:], Ain=None, method_deconvolution='oasis')

In [ ]:
# FIT
images = np.reshape(Yr.T, [T] + list(dims), order='F')
cnm = cnm.fit(images)

In [ ]:
A_or, C_or, srt = order_components(cnm.A,cnm.C)

In [ ]:
# view patches per layer
traces_fluo=nb_view_patches3d(Yr, A_or, C_or, cnm.b, cnm.f, dims, image_type='max',
                              max_projection=False,thr = 0.9)

In [ ]:
# view patches as maximum-projection
traces_fluo=nb_view_patches3d(Yr, A_or, C_or, cnm.b, cnm.f, dims, image_type='mean',
                              max_projection=True,axis=0,denoised_color='red')

In [ ]:
# STOP CLUSTER
cm.stop_server()